<a href="https://colab.research.google.com/github/thamyrissacamoto/MarvelAPI/blob/Thamyris%2FconsumindoAPI/client.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import hashlib
import time
import requests
from google.colab import userdata
import pandas as pd
from tqdm import tqdm

In [ ]:
BASE_URL = "https://gateway.marvel.com/v1/public/"
PUBLIC_KEY = userdata.get('public_key')
PRIVATE_KEY = userdata.get('private_key')

def get_auth():
    # Gera o hash md5 de autenticação

    ts = str(time.time())
    to_hash = ts + PRIVATE_KEY + PUBLIC_KEY
    hash_result = hashlib.md5(to_hash.encode()).hexdigest()
    return {
        "ts": ts,
        "apikey": PUBLIC_KEY,
        "hash": hash_result
    }


In [ ]:
etag_cache = {}

def marvel_request(endpoint, params_extra={}):
    # Faz uma requisição GET com cache ETag

    url = BASE_URL + endpoint
    auth_params = get_auth()
    params = auth_params.copy()
    params.update(params_extra)

    cache_key = f"{endpoint}_{params_extra}"
    headers = {}
    if cache_key in etag_cache:
        headers["If-None-Match"] = etag_cache[cache_key]["etag"]

    response = requests.get(url, params=params, headers=headers)

    if response.status_code == 304: # Evita refazer uma requisição
        return etag_cache[cache_key]["data"]
    elif response.status_code == 200:
        etag = response.headers.get("etag")
        data = response.json()
        etag_cache[cache_key] = {"etag": etag, "data": data}
        return data
    else:
        print(f"Erro {response.status_code}: {response.text}")
        return None

In [ ]:
def get_all_data(endpoint, relationships=[]):
    # Extrai dados do endpoint principal e relacionamentos

    first_response = marvel_request(endpoint, {"limit": 100, "offset": 0})
    if not first_response:
        print("Erro ao obter dados iniciais.")
        return None

    total = first_response["data"]["total"]
    all_results = first_response["data"]["results"]

    # Extrai todos os dados do endpoint principal
    for offset in tqdm(range(100, total, 100), desc=f"Baixando dados"):
        response = marvel_request(endpoint, {"limit": 100, "offset": offset})
        if response:
            all_results.extend(response["data"]["results"])
        time.sleep(0.3)

    # Busca relacionamentos para todos os IDs
    for item in tqdm(all_results, desc="Adicionando relacionamentos"):
        item_id = item["id"]
        for rel in relationships:
            rel_url = f"{endpoint}/{item_id}/{rel}"
            rel_response = marvel_request(rel_url)
            if rel_response:
                rel_ids = [r["id"] for r in rel_response["data"]["results"]]
                item[f"{rel}_ids"] = rel_ids
            else:
                item[f"{rel}_ids"] = []
            time.sleep(0.2)

    # Normaliza e salva em CSV
    df = pd.json_normalize(all_results)
    df.to_csv(f"{endpoint}.csv", index=False)
    return df

In [ ]:
# Endpoints e relacionamentos
endpoint_relationships = {
    "characters": ["comics", "events", "series", "stories"],
    "comics": ["creators"],
    "events": []
}

# Extrai cada endpoint
for endpoint, rels in endpoint_relationships.items():
    print(f"\nExtraindo dados de: {endpoint}")
    get_all_data(endpoint, relationships=rels)
